In [ ]:
import pandas as pd
import numpy as np
import scipy.optimize as sco
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, DateType
from pyspark.sql.functions import lit, last_day, col, array, explode, struct, udf, to_date, pandas_udf, PandasUDFType, sum as spark_sum, month, year

In [ ]:
spark = SparkSession.builder.appName('Rank_Mean').getOrCreate()
sc = spark.sparkContext
sc

In [ ]:
## por temas de replicación, siempre es mejor convertir el archivo a leer en formato csv

monthly_data = spark.read.parquet("../data/master/ophelia/data/OpheliaData/analytical_base_table")
index_vector = spark.read.csv("../data/raw/csv/unique_dateprice_vector.csv", header=True, inferSchema=True)

In [ ]:
#w = index_vector.limit(10)
#w.show()

In [ ]:
#import pandas as pd
#from scipy import stats

#multiply = pandas_udf(multiply_func, returnType=LongType())

#@pandas_udf('double')
#def cdf(v):
#    return pd.Series(stats.norm.cdf(v))


#asi = w.withColumn('cumulative_probability', cdf(w.MXWDU_Index))
#asi.toPandas()

In [ ]:
from datetime import datetime

def datetime_object(datetime_str):
    print(datetime_str)
    datetime_object = datetime.strptime(datetime_str, '%d/%m/%y').date()
    return datetime_object

datetime_object_udf = udf(f=datetime_object, returnType=DateType())

In [ ]:
index_vector_schema = index_vector.select("MXWDU_Index", datetime_object_udf(col("operation_date")).alias('operation_date'))
index_vector_schema.show(5, False)

In [ ]:
close_date_index = index_vector_schema.select("*", last_day(col("operation_date")).alias("close_date"))\
                                      .where(col("operation_date") == col("close_date")).orderBy(col("operation_date"))\
                                      .drop("close_date")
close_date_index.show(5, False)

In [ ]:
monthly_portfolio = monthly_data.select("*", last_day(col("operation_date")).alias("close_date"))\
                                .where(col("operation_date") == col("close_date")).drop("close_date")\
                                .orderBy("operation_date")

In [ ]:
index_portfolio_df = monthly_portfolio.join(close_date_index, on="operation_date", how="inner")
index_portfolio_df.show(5, False)

In [ ]:
monthly_data = index_portfolio_df.toPandas()
print("nuestros datos:", monthly_data.shape)
print("datos muestra:", _monthly_data.shape)

In [ ]:
## seleccionamos la primera columna y lo convertimos a vector, esta columna representa el indice que estamos analizando, 
## 'MXWDU_Index', la idea es medir esta columna con el resto, ya que las demás son acciones que forman parte de ése índice.

benchmark_month = monthly_data["MXWDU_Index"]

# nuestros datos
print(benchmark_month.shape)
benchmark_month

In [ ]:
## Calculamos el porcentaje de cambio de un día contra otro.
## fórmula: (t+1 / t)-1
## pseudo-código: (precio_hoy / precio_ayer)-1

pct_benchmark_month = benchmark_month.pct_change(1)
print(pct_benchmark_month.shape)

# nuestros datos
pct_benchmark_month

In [ ]:
## el vector de "percentage change" se convierte a un arreglo numpy de dimensión (147,)
## NOTA: en los arreglos (objetos) de tipo numpy.array preservan los valores, tanto por fila, como por columna,
##       el órden de los elementos, cómo un 'índice implícito'

pct_benchmark_month_array = np.array(pct_benchmark_month)
print(pct_benchmark_month_array.shape)
pct_benchmark_month_array

In [ ]:
## Lo mismo hacemos, pero para el resto de variables equity,
## seleccionamos la primera columna y lo convertimos a vector, esta columna representa el indice que estamos analizando, 
## 'MXWDU_Index', la idea es medir esta columna con el resto, ya que las demás son acciones que forman parte de ese índice.

investment_universe_month = monthly_data.drop(['operation_id', "operation_date", "MXWDU_Index"], axis = 1)

# nuestros datos
investment_universe_month.shape

In [ ]:
## (t+1 / t)-1
## (precio_hoy / precio_ayer)-1

pct_investment_month = investment_universe_month.pct_change(1)

# nuestros datos
pct_investment_month.shape

In [ ]:
## el vector de percentage change se convierte a un arreglo numpy de dimensión (147,70)

pct_investment_month_array = np.array(pct_investment_month)
#_pct_investment_month_array = np.array(_pct_investment_month)

# nuestros datos
print(pct_investment_month_array.shape)

In [ ]:
## creamos arreglos numpy con dimensiones X+1 = 148, rellenas de ceros, para ser imputados con nuevos vectores

up_month = np.zeros((pct_benchmark_month_array.shape[0]+1, 1))
down_month = np.zeros((pct_benchmark_month_array.shape[0]+1, 1))
up_move = np.zeros((pct_benchmark_month_array.shape[0]+1, pct_investment_month_array.shape[1]))
down_move = np.zeros((pct_benchmark_month_array.shape[0]+1, pct_investment_month_array.shape[1]))

print(up_month.shape)
print(down_month.shape)
print(up_move.shape)
print(down_move.shape)

In [ ]:
## rellenamos las matrices de ceros con valores que aprueben las condiciones, 
## se realiza una comparación dentro de los arreglos de porcentajes de cambio, 
## sí alguno de esos porcentajes es superior a 0, entonces entra a los arreglos 
## de movimientos positivos (incrementos), pero sí alguno es menor que 0, entonces
## el porcentaje se almacena en los arreglos de movimientos negativos (decrementos).

## Básicamente, se separan los porcentajes de cambio en dos matrices: 
## matriz de positivos cuando el porcentaje es > 0 
## matriz de negativos cuando el porcentaje es <= 0

size_benchmark_matrix = pct_benchmark_month_array.shape[0]
for i in range (1, size_benchmark_matrix):
    if pct_benchmark_month_array[i] > 0:
        up_month[i] = pct_benchmark_month_array[i]
        up_move[i] = pct_investment_month_array[i, 0:pct_investment_month_array.shape[1]]
    else:
        down_month[i] = pct_benchmark_month_array[i]
        down_move[i] = pct_investment_month_array[i, 0:pct_investment_month_array.shape[1]]

In [ ]:
## calculamos los vectores 'peor más alto' y 'mejor más alto'

np.seterr(divide='ignore', invalid='ignore')
greater_worse = down_move / down_month
greater_better = (up_move / up_month) * float(-1.0)

print(greater_worse.shape)
print(greater_better.shape)

In [ ]:
## ambos vectores los convertimos a pandas dataframes, y solo nos quedamos con los vectores que tengan valores != np.nan
## una de las ventajas de los pandas dataframes es que mantienen un ídince único por row, esto lo hace poder separarse, y juntarse
## en cuantos sub-conjuntos se requieran y siempre se podrá mantener un órden.

greater_worse_df = pd.DataFrameUtils(data=greater_worse).dropna()
greater_better_df = pd.DataFrameUtils(data=greater_better).dropna()

print(greater_worse_df.shape)
print(greater_better_df.shape)

In [ ]:
## calculamos ahora, la mediana acumulada con los pandas dataframes que construimos, 
## con un periodo mínimo (método expanding) de al menos 1 observación dada.

median_down = greater_worse_df.expanding().median()
median_up = greater_better_df.expanding().median()

print(median_down.shape)
print(median_up.shape)

In [ ]:
# se transponen ambos pandas df por la columna periodos, columna que almacena números no consecutivos desde 1 hasta 147

down_transpose = median_down.T
up_transpose = median_up.T

print(down_transpose.shape)
print(up_transpose.shape)

In [ ]:
# se rankean los resultados (top 10) entre las fechas cierre (periodo) y se vuelve a transponer la tabla ranked_down

ranked_down = down_transpose.rank(method='first')
transpose_ranked_down = ranked_down.T

print(ranked_down.shape)
print(transpose_ranked_down.shape)

In [ ]:
# se rankean los resultados (top 10) entre las fechas cierre (periodo) y se vuelve a transponer la tabla ranked_up

ranked_up = up_transpose.rank(method='first')
transpose_ranked_up = ranked_up.T

print(ranked_up.shape)
print(transpose_ranked_up.shape)

In [ ]:
## se añade variable 'label' con la idea de que al juntar ambos dataframes se puedan distinguir los 'worse' de los 'better'
## y se unen ambos dataframes con la etiqueta creada, se usó el método 'insert' por lo que no se deberá correr de nuevo, una
## vez ejecutado ya que fallará por duplicidad de columnas.

worse_better_df = pd.concat([transpose_ranked_up, transpose_ranked_down]).sort_index()
worse_better_df.shape

In [ ]:
## se crea un índice 'closing_id' para cada registro, éste corre de [1:N] 
## con la idea de etiquetar el id del mes de registro de cierre,
## de la misma forma que lo anterior, NO se deberá ejecutar de nuevo; una vez hecho.

worse_better_df['closing_id'] = range(1, len(worse_better_df) + 1)
worse_better_df.shape

In [ ]:
## se transponen ambos dataframes, de antes tener una dimensión (68, 70), es decir; 
## 68 registros i.e. 'Rows' (variables)
## 70 columnas fijas (a menos que sea añadido otro asset desde el csv inicial)

## a tener una dimensión 'transpuesta' (invertída sí querés...) de (70, 68), es decir;
## 70 registros i.e. 'Rows' fijos (a menos que sea añadido otro asset desde el csv inicial)
## 68 columnas (variables!!!)

In [ ]:
## de pandas dataframes, una vez separados en dos conjuntos ['worse', 'better'],
## creamos por separado dos spark dataframes.

worse_better = spark.createDataFrame(worse_better_df)
worse_better.show(5, False)

In [ ]:
## se crea un generador "shape long format", una lista con iteraciones, esta lista trabajará con dos variables principales,
## 1-. la variable 'equity_index', será la que contenga los 'id' de los activos
## 2-. la variable 'median_down', será la que contenga la mediana acumulada por cada activo.
## Se mantendrá a lo largo de la transformación 1 columna fija; 'closing_id',
## closing_id: variable que indica el mes de cierre y reporte de precio

def panel_format(df, pivot_col):
    
    piv_col = [pivot_col]
    df_types = df.dtypes
    cols, dtype = zip(*((c, t) for (c, t) in df_types if c not in piv_col))
    
    if len(set(dtype)) == 1:
        ValueError("Columns not the same data type...")
    
    generator_explode = explode(array([
        struct(lit(c).alias("asset_id"), col(c).alias("rank")) for c in cols
    ])).alias("column_explode")
    
    panel_df = df.select(piv_col + [generator_explode])\
                 .select(piv_col + ["column_explode.asset_id", "column_explode.rank"])

    return panel_df

In [ ]:
## se crea nuevo spark-dataframe donde solo se mostrará por partición ['closing_id'] 
## el top 10 mejores meses donde tuvo menos malos que el resto de los registros; ["top_rank"].

asset_ranking_df = panel_format(worse_better, "closing_id")
asset_ranking_df.show(5, False)

In [ ]:
asset_rank_10 = asset_ranking_df.where(col("rank") <= 10).orderBy("closing_id", "rank").toPandas()

# A partir de aqui se escribe en formato csv para trabajar con Pandas

In [ ]:
## lo que se pretende ahora, es obtener una matriz de dimensión (146, 10), es decir, 
## tener en cada row las fechas de cierre [closing_id],
## en cada columna (header) el número de ranking top 10 [top_rank],
## y en cada campo, el id del activo [asset_id].

pandas_df = asset_rank_10.astype({'asset_id':'int32'})
newselect = asset_rank_10[["closing_id", "asset_id"]]
num_of_assets = 10

indexed = np.zeros((investment_universe_month.shape[0], num_of_assets))

for q in range(1,investment_universe_month.shape[0]):
    selection = newselect.loc[pandas_df["closing_id"]==q]
    newselect_transpose = selection.T
    newpdf = newselect_transpose['asset_id':].head()
    indexed[q] = newpdf

In [ ]:
indexed = indexed[1:investment_universe_month.shape[0]+1]
index_row = indexed.astype(np.int64)
newrow = [0]*num_of_assets #np.zeros((1,num_of_assets))
index_row = np.vstack([index_row,newrow])
portfolio = np.zeros((investment_universe_month.shape[0], num_of_assets))

for r in range(0,investment_universe_month.shape[0]-1):
    s = r+1
    columns = index_row[s]
    portfolio[s] = pct_investment_month_array[s,[columns]]

In [ ]:
means = portfolio[1:investment_universe_month.shape[0]-1]
performance = np.dot(portfolio,(1/num_of_assets))
returns = np.zeros((investment_universe_month.shape[0]-1,1))
equalw = np.zeros((investment_universe_month.shape[0]-1,1))
eweights = 1/(pct_investment_month_array.shape[1])
eweighted = np.dot(pct_investment_month_array,eweights)

for x in range (1,investment_universe_month.shape[0]):
    returns[x-1] = sum(performance[x])
    equalw[x-1] = sum(eweighted[x])

In [ ]:
beg = 100
start = 100
commence = 100
bench = pct_benchmark_month_array[1:investment_universe_month.shape[0]]
bmk = np.zeros((investment_universe_month.shape[0],))
port = np.zeros((investment_universe_month.shape[0]-1,))
ew = np.zeros((investment_universe_month.shape[0]-1,))

for i in range (0,investment_universe_month.shape[0]-1):
    bmk[i] = beg*(1+bench[i])
    beg = bmk[i]
    port[i] = start*(1+returns[i])
    start = port[i]
    ew[i] = commence*(1+equalw[i])
    commence = ew[i]

In [ ]:
plt.plot(bmk[0:investment_universe_month.shape[0]-1])
plt.plot(port)
plt.plot(ew)
input_assets = index_row[index_row.shape[0]-2]
input_assets1 = pct_investment_month_array[:, input_assets]
dataframe = pd.DataFrameUtils(input_assets1)
input_assets1 = dataframe.dropna()
mean_returns = input_assets1.mean()
covar_matrix = input_assets1.cov()
tickers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

def calc_portfolio_perf(weights, mean_returns, cov, rf):
    portfolio_return = np.sum(mean_returns * weights) * 12
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov, weights))) * np.sqrt(252)
    sharpe_ratio = (portfolio_return - rf) / portfolio_std
    return portfolio_return, portfolio_std, sharpe_ratio

In [ ]:
def simulate_random_portfolios(num_portfolios, mean_returns, cov, rf):
    results_matrix = np.zeros((len(mean_returns)+3, num_portfolios))
    for i in range(num_portfolios):
        weights = np.random.random(len(mean_returns))
        weights /= np.sum(weights)
        portfolio_return, portfolio_std, sharpe_ratio = calc_portfolio_perf(weights, mean_returns, cov, rf)
        results_matrix[0,i] = portfolio_return
        results_matrix[1,i] = portfolio_std
        results_matrix[2,i] = sharpe_ratio

        for j in range(len(weights)):
            results_matrix[j+3,i] = weights[j]

    results_df = pd.DataFrameUtils(results_matrix.T,columns=['ret','stdev','sharpe'] + [ticker for ticker in tickers])

    return results_df

In [ ]:
mean_returns = input_assets1.mean()
cov = input_assets1.cov()
num_portfolios = 10000
rf = 0.0
results_frame = simulate_random_portfolios(num_portfolios, mean_returns, cov, rf)

max_sharpe_port = results_frame.iloc[results_frame['sharpe'].idxmax()]

min_vol_port = results_frame.iloc[results_frame['stdev'].idxmin()]

max_return_port = results_frame.iloc[results_frame['ret'].idxmax()]

plt.subplots(figsize=(15,10))
plt.scatter(results_frame.stdev,results_frame.ret, c=results_frame.sharpe, cmap='RdYlBu')
plt.xlabel('Standard Deviation')
plt.ylabel('Returns')
plt.colorbar()
plt.scatter(max_sharpe_port['stdev'], max_sharpe_port['ret'], marker=(5,1,0), color='r', s=500)
plt.scatter(min_vol_port['stdev'], min_vol_port['ret'], marker=(5,1,0), color='g', s=500)
plt.scatter(max_return_port['stdev'], max_return_port['ret'], marker=(5,1,0), color='y', s=500)
plt.show()